In [ ]:
%load_ext autoreload
%autoreload 2

import json5
import os
import json
if "__file__" not in globals():
    __file__ = os.path.abspath(".")
print("Running in file:", __file__)
test_none_path=os.path.join(__file__, 'data','MatSciInstruct_test_none.json')
test_txt_path=os.path.join(__file__,'data', 'MatSciInstruct_test_txt.json')

with open(test_txt_path, 'r') as f:
    # Load the JSON5 data from the file
    data= json.load(f)
with open(test_none_path, 'r') as f:
    # Load the JSON5 data from the file
    none_data= json.load(f)

print("Total number of data points:", len(data))
print("Total number of data points with None in 'answer':", len(none_data))

Running in file: /u01/mengpengyu/dataPipe
Data path: /u01/mengpengyu/dataPipe/data/MatSciInstruct.json
None data path: /u01/mengpengyu/dataPipe/data/MatSciInstruct_none.json
Total number of data points: 2307
Total number of data points with None in 'answer': 471


### 翻译部分问题，用于人工判别

In [ ]:
# from deep_translator import BaiduTranslator
# import json

# # 创建翻译器并进行翻译
# translator = BaiduTranslator(source='en', 
#                                   target='zh',
#                                   appid="20200702000511635",
#                                   appkey='QkuPegRc_zjsNH0Ytn52'
#                                   )
# # 翻译问题，存储到一个json文件中，后续方便查找
# translate_path=os.path.join(__file__, 'data','MatSciInstruct_translate.json')
# translate_list = []
# i=0
# for signal_data in data[:200]:
#     raw_problem = signal_data['metadata']['instruction']
#     translated_problem = translator.translate(raw_problem)
#     data_id = signal_data['id']
#     translate_list.append({
#         "id": data_id,
#         "raw_problem": raw_problem,
#         "translated_problem": translated_problem
#     })
#     if i % 10 == 0:
#         print(f"Translated {i} problems")
#     i+=1
# print("Translated problems:", translate_list)
# with open(translate_path, 'w') as f:
#     json.dump(translate_list, f, indent=4, ensure_ascii=False)

In [ ]:

translate_path=os.path.join(__file__, 'data','MatSciInstruct_translate.json')
with open(translate_path, 'r') as f:
    # Load the JSON5 data from the file
    translate_data= json5.load(f)
from model import call_openai,call_huoshan
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor

prompt_path=os.path.join(__file__, 'prompts','division.txt')
with open(prompt_path, 'r') as f:
    raw_prompt_template = f.read()

model_list= ["r1", "4o", "doubao"]
# model_name = "doubao"
##  "r1"
##  "4o"
##  "doubao"
def process_item(signal_data_item,trans_problem_item,model_name):
    instruction = signal_data_item['metadata']['instruction']
    input_val = signal_data_item['metadata']['input'] 
    output_val = signal_data_item['metadata']['output']

    prompt = raw_prompt_template.replace("{{instruction}}", str(instruction)) \
                                .replace("{{input}}", str(input_val)) \
                                .replace("{{output}}", str(output_val))
    if model_name == "4o":
        _,content = call_openai(prompt)
    else:
        _, content = call_huoshan(prompt,model_name)
    problem_id = signal_data_item['id']
    return content, trans_problem_item,problem_id
import json
model_answer_path=os.path.join(__file__, 'data','MatSciInstruct_answer.json')
model_answer={}
with open(model_answer_path, 'r') as f:
    # Load the JSON5 data from the file
    try:
        model_answer = json.load(f)
    except json.JSONDecodeError:
        print("JSONDecodeError: The file is empty or not a valid JSON.")
import json_repair
end=200
def run_use_model(model_name):
    with ThreadPoolExecutor(max_workers=64) as executor:
        future_to_data = {}
        for signal_data_item, trans_problem_item in zip(data[0:end], translate_data[0:end]):
            future = executor.submit(process_item, signal_data_item, trans_problem_item,model_name)
            future_to_data[future] = trans_problem_item

        for future in concurrent.futures.as_completed(future_to_data):
            try:
                content, trans_problem_result ,problem_id= future.result() # 获取 process_item 的返回值
                res_json= json_repair.repair_json(content,return_objects=True)
                if problem_id in model_answer:
                    model_answer[problem_id][model_name] = res_json
                else:
                    anser={
                        "raw_problem":trans_problem_result['raw_problem'],
                        "translated_problem" :trans_problem_result['translated_problem'],
                        model_name: res_json
                    }
                    model_answer[problem_id]=anser
                # 打印结果
                print(content)
                print(f"ID: {problem_id}")
                print(f"raw problem: {trans_problem_result['raw_problem']}")
                print(f"translated problem: {trans_problem_result['translated_problem']}")
                print("-" * 50)
            except Exception as exc:
                failed_trans_problem = future_to_data[future]
                print(f"Item with raw problem '{failed_trans_problem['raw_problem']}' generated an exception: {exc}")

# for model_name in model_list:
#     print(f"Running model: {model_name}")
#     run_use_model(model_name)
with ThreadPoolExecutor(max_workers=len(model_list)) as outer_executor:
    for model_name in model_list:
        outer_executor.submit(run_use_model, model_name)
with open(model_answer_path, 'w') as f:
    json.dump(model_answer, f, indent=4, ensure_ascii=False)
print("Model answers saved to:", model_answer_path)

```json
{
  "category": "KNOWLEDGE",
  "reason": "该问题的答案是基于上下文中的明确陈述内容，属于客观事实，无需推理即可得出。"
}
```
ID: 738abd95be987a0f02e0675d3f3387b1
raw problem: What is the corollary of the study?
translated problem: 这项研究的推论是什么？
--------------------------------------------------
```json
{
  "category": "KNOWLEDGE",
  "reason": "问题的答案可以直接从上下文中提取，属于客观事实性知识，无需复杂推理。"
}
```
ID: 5c65bef00762816a5d9de9ecf98b3bf2
raw problem: What is the valence of Eu in EuPd$_2$(Si$_{1-x}$Ge$_x$)$_2$ for x = 0.2?
translated problem: 当x=0.2时，EuPd$_2$（Si$_{1-x}$Ge$_x$）$_2$中Eu的化合价是多少？
--------------------------------------------------
```json
{
  "category": "KNOWLEDGE",
  "reason": "该问题要求提取上下文中明确描述的理论内容，属于客观事实性知识，无需复杂推理。"
}
```
ID: 82b284724f7a1e32d4550c57edc81339
raw problem: What is the analytical theory developed in the text?
translated problem: 本文中发展的分析理论是什么？
--------------------------------------------------
```json
{
  "category": "KNOWLEDGE",
  "reason": "问题的答案是直接从上下文中提取的三个客观事实，无需复杂的推理或分析。"
}
```
ID: e0ec17710f199f876c5